Build a core data model for the food consumed

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.default.daily_meal_clean AS
WITH deduped_food AS (
  SELECT
    *,
    ROW_NUMBER() OVER (
      PARTITION BY
        date,
        meal_type,
        title
      ORDER BY date DESC
    ) AS rn
  FROM workspace.default.food
)
SELECT
  year(date) * 10000 + month(date) * 100 + dayofmonth(date) AS sk_date,
  date,
  meal_type,
  title,
  brand,
  amount_in_grams,
  calories,
  carbs,
  carbs_fiber,
  carbs_sugar,
  cholesterol,
  fat,
  fat_saturated,
  fat_unsaturated,
  potassium,
  protein,
  sodium,
  hash(date, meal_type, title) AS row_hash,
  current_timestamp() AS load_datetime
FROM deduped_food
WHERE 1=1 
AND rn = 1 
AND date IS NOT NULL
AND meal_type IS NOT NULL
AND title IS NOT NULL

In [0]:
%sql
MERGE INTO workspace.silver.daily_meal AS target
USING workspace.default.daily_meal_clean AS source
ON target.row_hash = source.row_hash
WHEN MATCHED THEN
  UPDATE SET
    target.sk_date = source.sk_date,
    target.date = source.date,
    target.meal_type = source.meal_type,
    target.food_item = source.title,
    target.brand = source.brand,
    target.amount_in_grams = source.amount_in_grams,
    target.calories = source.calories,
    target.carbs = source.carbs,
    target.carbs_fiber = source.carbs_fiber,
    target.carbs_sugar = source.carbs_sugar,
    target.cholesterol = source.cholesterol,
    target.fat = source.fat,
    target.fat_saturated = source.fat_saturated,
    target.fat_unsaturated = source.fat_unsaturated,
    target.potassium = source.potassium,
    target.protein = source.protein,
    target.sodium = source.sodium,
    target.update_datetime = current_timestamp()
WHEN NOT MATCHED THEN
  INSERT (
    sk_date,
    date,
    meal_type,
    food_item,
    brand,
    amount_in_grams,
    calories,
    carbs,
    carbs_fiber,
    carbs_sugar,
    cholesterol,
    fat,
    fat_saturated,
    fat_unsaturated,
    potassium,
    protein,
    sodium,
    row_hash,
    load_datetime,
    update_datetime
  )
  VALUES (
    source.sk_date,
    source.date,
    source.meal_type,
    source.title,
    source.brand,
    source.amount_in_grams,
    source.calories,
    source.carbs,
    source.carbs_fiber,
    source.carbs_sugar,
    source.cholesterol,
    source.fat,
    source.fat_saturated,
    source.fat_unsaturated,
    source.potassium,
    source.protein,
    source.sodium,
    source.row_hash,
    current_timestamp(),
    current_timestamp()
  )